In [19]:
#Business Search      URL -- 'https://api.yelp.com/v3/businesses/search'
#Business Match       URL -- 'https://api.yelp.com/v3/businesses/matches'
#Phone Search         URL -- 'https://api.yelp.com/v3/businesses/search/phone'

#Business Details     URL -- 'https://api.yelp.com/v3/businesses/{id}'
#Business Reviews     URL -- 'https://api.yelp.com/v3/businesses/{id}/reviews'
#Delivary                     https://api.yelp.com/v3/transactions/{transaction_type}/search
#Businesses, Total, Region

# Import the modules
import requests
import json
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import requests 
import time
import json
import seaborn 

# connect to API key
from config import yelpkey

# Define API Key, Endpoint, and Header
API_KEY = yelpkey
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
HEADERS = {'Authorization': 'bearer %s' % API_KEY}

# Let's first do an easy call to see how we find the total # of restaurants in a zip code

# Define parameters of  search: chinese restaurants in zipcode 10007
PARAMETERS = {'term': 'chinese+food',
              'location': '10001',
               'limit': 50}

# Make a request to the Yelp API
response = requests.get(url = ENDPOINT,
                        params = PARAMETERS,
                        headers = HEADERS)

# Convert the JSON String
business_data = response.json()


# print some basic data on the response
num_of_results = business_data['total']
biz = []

count_in_zip = 0
for restaurant in business_data['businesses']:
    if restaurant['location']['zip_code'] == PARAMETERS['location']:
        count_in_zip += 1
        biz.append(restaurant['alias'])

print(f'{num_of_results} search results total near {PARAMETERS["location"]}')
print(f'There are {count_in_zip} "{PARAMETERS["term"].replace("+", " ")}" businesses directly in {PARAMETERS["location"]}:\n')
print(*biz, sep='\n')

719 search results total near 10001
There are 15 "chinese food" businesses directly in 10001:

golden-city-chinese-restaurant-new-york
stick-to-my-pot-potsticker-new-york
woorijip-authentic-korean-food-new-york
grand-sichuan-new-york-12
kobeyaki-new-york-2
asura-asian-bistro-new-york-4
izakaya-mew-new-york-3
mr-bing-new-york-9
barn-joo-35-new-york
jun-men-ramen-bar-new-york
bcd-tofu-house-new-york-3
mings-restaurant-new-york-2
panda-express-new-york-8
ichiran-midtown-new-york
new-dynasty-chinese-restaurant-new-york


In [4]:
# we can see in the response above that there are three keys, one of which is "total"

# Print the value in the "total" key to see how many Chinese restaurants are in that zip code
print(business_data['total'])

227


In [ ]:
# so now let's apply that approach with a list of zip codes instead of just one

In [10]:
# import NY zip codes list from CSV & convert into dataframe
# note, to test, I'm using an abbreciated version of our zip code list 

zipcodes_data = "clean_operational_ny_zips_SHORT.csv" 
zipcodes_df = pd.read_csv(zipcodes_data)
zipcodes_df

,Zip Code
0,10001
1,10002
2,10003
3,10004
4,10005
5,10006
6,10007


In [11]:
# Add column to hold number of Chinese restaurants
zipcodes_df["# Chinese Rest"] = ""
zipcodes_df


,Zip Code,# Chinese Rest
0,10001,
1,10002,
2,10003,
3,10004,
4,10005,
5,10006,
6,10007,


In [22]:
# create a loop that looks up the zip in each row and adds total # Chinese rest to the column

for index, row in zipcodes_df.iterrows():    
    zip = row['Zip Code']
    PARAMETERS = {'term': 'chinese+food',
                  'location': zip,
                  'limit': 50}
    try:
        response = requests.get(url = ENDPOINT,
                                params = PARAMETERS,
                                headers = HEADERS)
        business_data = response.json()
        print(f"Finding restaurant info for zip code: {zip}")
        count_in_zip = 0
        for restaurant in business_data['businesses']:
            if restaurant['location']['zip_code'] == str(PARAMETERS['location']):
                count_in_zip += 1
        zipcodes_df.loc[index, "# Chinese Rest"] = count_in_zip
    
    except json.decoder.JSONDecodeError:
        print("Missing field/result... skipping.")
        
    print("-------------------------------------------")

Finding restaurant info for zip code: 10001
-------------------------------------------
Finding restaurant info for zip code: 10002
-------------------------------------------
Finding restaurant info for zip code: 10003
-------------------------------------------
Finding restaurant info for zip code: 10004
-------------------------------------------
Finding restaurant info for zip code: 10005
-------------------------------------------
Finding restaurant info for zip code: 10006
-------------------------------------------
Finding restaurant info for zip code: 10007
-------------------------------------------


In [23]:
# print the data frame, which should now include the Chinese restaurant count by zip
zipcodes_df


,Zip Code,# Chinese Rest
0,10001,6
1,10002,9
2,10003,16
3,10004,4
4,10005,4
5,10006,7
6,10007,4
